Autoloader is built on top of Spark structure streaming.

### Incremental Data Ingestion

In [0]:
# Parameter widget to show a box
dbutils.widgets.text("src","")

In [0]:
src_value = dbutils.widgets.get("src")
src_value

In [0]:
df = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format","csv")\
    .option("cloudFiles.schemaLocation",f"/Volumes/workspace/bronze/bronzevolume/{src_value}/checkpoint")\
    .option("cloudFiles.schemaEvolutionMode","rescue")\
    .load(f"/Volumes/workspace/raw/rawvolume/rawdata/{src_value}/")
    

In [0]:
# trigger(once=True) will terminate your cluster
# checkpointLocation should be same as the value of cloudFiles.schemaLocation
df.writeStream.format("delta")\
    .outputMode("append")\
    .trigger(once=True)\
    .option("checkpointLocation",f"/Volumes/workspace/bronze/bronzevolume/{src_value}/checkpoint")\
    .option("path",f"/Volumes/workspace/bronze/bronzevolume/{src_value}/data")\
    .start()

In [0]:
%sql
SELECT * FROM DELTA.`/Volumes/workspace/bronze/bronzevolume/bookings/data`;